# Addition of a Neural Network Model

After creating the previous three models for consideration, I decided to go back and take a look at the possibility of adding a fourth model. I wanted to see if I can use a model with Deep Neural Networks for a more realistic assessment of housing prices. I was able to reach my goal of coming under $200000, but I believe that I can do even better with the use of a neural network. I am going to bring in my cleaned data so that I can go right into the modeling process. 

# Importing Libraries

In [1]:
import tensorflow as tf
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
np.random.seed(9)
random.seed(9)
tf.random.set_seed(9)

# Bringing in the Cleaned Dataset

In [3]:
#bringing in the cleaned
infile = open("Data/cleaned_data.pickle",'rb')
df = pickle.load(infile)
infile.close()

In [4]:
df.head()

,price,bedrooms,bathrooms,sqft_living,floors,waterfront,sqft_above,sqft_basement,sqft_living15,log_yard,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,365000.0,4.0,2.0,2070.0,2.0,0,2070.0,0,2390,8.969287,...,0,0,0,0,0,0,0,0,0,0
1,865000.0,5.0,3.0,2900.0,1.0,0,1830.0,1070,2370,8.250620,...,0,0,0,0,0,0,0,0,0,0
2,1038000.0,4.0,2.0,3770.0,2.0,0,3770.0,0,3710,9.105868,...,0,0,0,0,0,0,0,0,0,0
3,1490000.0,3.0,4.0,4560.0,2.0,0,4560.0,0,4050,9.419628,...,0,0,0,0,0,0,0,0,0,0
4,711000.0,3.0,2.0,2550.0,2.0,0,2550.0,0,2250,8.318986,...,0,0,0,0,0,0,0,0,0,0


# Preparing the Data

In [5]:
#dropping the dependent variable
dvariables = df.iloc[:, 1:]

#this here will become the SHAPE variable in the function below
number_of_columns = len(dvariables.columns)

#isolating the target variable
target = df.iloc[:,0]

In [6]:
#implementing train test split
X_fulltrain, X_fulltest, y_fulltrain, y_fulltest = train_test_split(dvariables, target, random_state = 9, test_size = .2)

#splitting again for a validation set
X_train, X_valid, y_train, y_valid = train_test_split(X_fulltrain, y_fulltrain, random_state = 9, test_size = .2)


# Model Creation

In [7]:
def create_model(optimizer = 'adam', num_layers = 1, neurons = 30, drop_out= .2, input_shape = 106, learning_rate = 3e-4):
    """This function takes an optimizer, number of layers, and input_shape. It will create
    and compile the model so that it can be ready to be fit in the next step."""
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape = (input_shape,)))
    for layer in range(num_layers):
        model.add(tf.keras.layers.Dense(neurons, activation = 'relu'))
    #This dropout should hopefully be preventing overfitting
    model.add(tf.keras.layers.Dropout(drop_out))
    model.add(tf.keras.layers.Activation('linear'))
    model.add(tf.keras.layers.Dense(1))
    
    
    model.compile(optimizer, loss = 'mse', metrics = ['mse'])
    return model

In [8]:
model_1 = create_model(num_layers = 1)

In [9]:
#fitting the model
history = model_1.fit(X_train, y_train, epochs = 80, 
                      #utilizing our validation set to test/prevent overfitting
                      validation_data = (X_valid, y_valid)) 



Epoch 1/80
346/346 [==============================] - 0s 1ms/step - loss: 427578458112.0000 - mse: 427578458112.0000 - val_loss: 383302434816.0000 - val_mse: 383302434816.0000
Epoch 2/80
346/346 [==============================] - 0s 963us/step - loss: 377481003008.0000 - mse: 377481003008.0000 - val_loss: 316028649472.0000 - val_mse: 316028649472.0000
Epoch 3/80
346/346 [==============================] - 0s 806us/step - loss: 297875111936.0000 - mse: 297875111936.0000 - val_loss: 232643461120.0000 - val_mse: 232643461120.0000
Epoch 4/80
346/346 [==============================] - 0s 811us/step - loss: 215591829504.0000 - mse: 215591829504.0000 - val_loss: 157850714112.0000 - val_mse: 157850714112.0000
Epoch 5/80
346/346 [==============================] - 0s 937us/step - loss: 150588375040.0000 - mse: 150588375040.0000 - val_loss: 106434174976.0000 - val_mse: 106434174976.0000
Epoch 6/80
346/346 [==============================] - 0s 1ms/step - loss: 108495872000.0000 - mse: 108495872000.

Epoch 48/80
346/346 [==============================] - 0s 815us/step - loss: 79700746240.0000 - mse: 79700746240.0000 - val_loss: 65656176640.0000 - val_mse: 65656176640.0000
Epoch 49/80
346/346 [==============================] - 0s 807us/step - loss: 80272236544.0000 - mse: 80272236544.0000 - val_loss: 65657122816.0000 - val_mse: 65657122816.0000
Epoch 50/80
346/346 [==============================] - 0s 769us/step - loss: 81593925632.0000 - mse: 81593925632.0000 - val_loss: 65659469824.0000 - val_mse: 65659469824.0000
Epoch 51/80
346/346 [==============================] - 0s 782us/step - loss: 80642744320.0000 - mse: 80642744320.0000 - val_loss: 65666236416.0000 - val_mse: 65666236416.0000
Epoch 52/80
346/346 [==============================] - 0s 837us/step - loss: 79390244864.0000 - mse: 79390244864.0000 - val_loss: 65681113088.0000 - val_mse: 65681113088.0000
Epoch 53/80
346/346 [==============================] - 0s 750us/step - loss: 81966424064.0000 - mse: 81966424064.0000 - val_l

# Evaluation

In [10]:
print('Evaluation at Epoch 80:\n')
for key in history.history:
    print(key + ':' + str(history.history[key][-1]) + '\n')

print('RMSE :' + str(np.sqrt(history.history['val_mse'][-1])))

Evaluation at Epoch 80:

loss:80479903744.0

mse:80479903744.0

val_loss:65825914880.0

val_mse:65825914880.0

RMSE :256565.6151552659


In terms of the RMSE the model has not crossed the threshold of the goal RMSE. But the good news is that there does not seem to be evidence of overfitting. I will continue on with the nn modeling process and try to lower the error by tinkering manually with the parameters.

# Second Model Attempt

For an attempt at improvement, I am going to try another model with more layers to see how that would compare to the first attempt. 

In [11]:
#I am increasing the number of layers in this model
model_2 = create_model(num_layers = 12)

In [12]:
history_2 = model_2.fit(X_train, y_train, epochs = 80, validation_data=(X_valid, y_valid))

Epoch 1/80
346/346 [==============================] - 1s 2ms/step - loss: 143858401280.0000 - mse: 143858401280.0000 - val_loss: 65858732032.0000 - val_mse: 65858732032.0000
Epoch 2/80
346/346 [==============================] - 0s 1ms/step - loss: 85689106432.0000 - mse: 85689106432.0000 - val_loss: 68905230336.0000 - val_mse: 68905230336.0000
Epoch 3/80
346/346 [==============================] - 0s 1ms/step - loss: 84045225984.0000 - mse: 84045225984.0000 - val_loss: 65371648000.0000 - val_mse: 65371648000.0000
Epoch 4/80
346/346 [==============================] - 0s 1ms/step - loss: 82223259648.0000 - mse: 82223259648.0000 - val_loss: 65959161856.0000 - val_mse: 65959161856.0000
Epoch 5/80
346/346 [==============================] - 0s 1ms/step - loss: 80372695040.0000 - mse: 80372695040.0000 - val_loss: 70882213888.0000 - val_mse: 70882213888.0000
Epoch 6/80
346/346 [==============================] - 0s 1ms/step - loss: 78023237632.0000 - mse: 78023237632.0000 - val_loss: 71993516032

346/346 [==============================] - 0s 1ms/step - loss: 59613036544.0000 - mse: 59613036544.0000 - val_loss: 41280327680.0000 - val_mse: 41280327680.0000
Epoch 49/80
346/346 [==============================] - 0s 1ms/step - loss: 55575535616.0000 - mse: 55575535616.0000 - val_loss: 40885018624.0000 - val_mse: 40885018624.0000
Epoch 50/80
346/346 [==============================] - 0s 1ms/step - loss: 56002883584.0000 - mse: 56002883584.0000 - val_loss: 41237901312.0000 - val_mse: 41237901312.0000
Epoch 51/80
346/346 [==============================] - 0s 1ms/step - loss: 54529191936.0000 - mse: 54529191936.0000 - val_loss: 37793935360.0000 - val_mse: 37793935360.0000
Epoch 52/80
346/346 [==============================] - 0s 1ms/step - loss: 54906830848.0000 - mse: 54906830848.0000 - val_loss: 48173109248.0000 - val_mse: 48173109248.0000
Epoch 53/80
346/346 [==============================] - 0s 991us/step - loss: 55747051520.0000 - mse: 55747051520.0000 - val_loss: 39492460544.0000 

In [13]:
print('Evaluation at Epoch 80:\n')
for key in history_2.history:
    print(key + ':' + str(history_2.history[key][-1]) + '\n')
    
print('RMSE :' + str(np.sqrt(history_2.history['val_mse'][-1])))

Evaluation at Epoch 80:

loss:48876310528.0

mse:48876310528.0

val_loss:30303090688.0

val_mse:30303090688.0

RMSE :174077.8293982321


The error here is reduced from the last attempt. This tells us that more layers might have been needed to get a better estimate of the target. The error measured in the validation set gets us over the threshold and achieves our goal of less than 200000. It does not however compare to the original models created. I think the next best step would be to incorporate randomized search to find optimal parameters.

# Implementing RandomizedSearch

In [14]:
#create a base model for Gridsearch
model = KerasRegressor(build_fn=create_model)

In [15]:
neurons = tuple(range(20, 101))
drop_out = (.1, .2, .5, .8, .9)
epochs = (70, 80, 90, 100, 110)
num_layers = tuple(range(2, 19))
learning_rate = (3e-2, 3e-4)
optimizer = ('Adam', 'RMSprop')
batch_size = tuple(range(10, 51))
parameters = {'neurons': neurons, 'drop_out': drop_out, 'num_layers': num_layers,
             'optimizer': optimizer, 'epochs':epochs, 'learning_rate': learning_rate}

In [16]:
random_model = RandomizedSearchCV(model, parameters, cv = 3, verbose = 1)

In [17]:
history_3 = random_model.fit(X_train, y_train, validation_data = (X_valid, y_valid),
                             #I'm going to try to use Early Stopping. It will hopefully
                             #get a lower score, and maybe will lower the total runtime.
                            callbacks = [tf.keras.callbacks.EarlyStopping(patience = 20)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/70


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


231/231 [==============================] - 0s 1ms/step - loss: 255763922944.0000 - mse: 255763922944.0000 - val_loss: 81942757376.0000 - val_mse: 81942757376.0000
Epoch 2/70
231/231 [==============================] - 0s 818us/step - loss: 143466020864.0000 - mse: 143466020864.0000 - val_loss: 75034673152.0000 - val_mse: 75034673152.0000
Epoch 3/70
231/231 [==============================] - 0s 814us/step - loss: 143018344448.0000 - mse: 143018344448.0000 - val_loss: 76202106880.0000 - val_mse: 76202106880.0000
Epoch 4/70
231/231 [==============================] - 0s 815us/step - loss: 138700095488.0000 - mse: 138700095488.0000 - val_loss: 78732337152.0000 - val_mse: 78732337152.0000
Epoch 5/70
231/231 [==============================] - 0s 942us/step - loss: 144217931776.0000 - mse: 144217931776.0000 - val_loss: 74816061440.0000 - val_mse: 74816061440.0000
Epoch 6/70
231/231 [==============================] - 0s 826us/step - loss: 141957627904.0000 - mse: 141957627904.0000 - val_loss: 77

Epoch 11/70
231/231 [==============================] - 0s 1ms/step - loss: 148054441984.0000 - mse: 148054441984.0000 - val_loss: 80946995200.0000 - val_mse: 80946995200.0000
Epoch 12/70
231/231 [==============================] - 0s 1ms/step - loss: 145269817344.0000 - mse: 145269817344.0000 - val_loss: 78486257664.0000 - val_mse: 78486257664.0000
Epoch 13/70
231/231 [==============================] - 0s 1ms/step - loss: 148045660160.0000 - mse: 148045660160.0000 - val_loss: 76092342272.0000 - val_mse: 76092342272.0000
Epoch 14/70
231/231 [==============================] - 0s 1ms/step - loss: 138333339648.0000 - mse: 138333339648.0000 - val_loss: 76198789120.0000 - val_mse: 76198789120.0000
Epoch 15/70
231/231 [==============================] - 0s 1ms/step - loss: 140118212608.0000 - mse: 140118212608.0000 - val_loss: 83236798464.0000 - val_mse: 83236798464.0000
Epoch 16/70
231/231 [==============================] - 0s 1ms/step - loss: 150738092032.0000 - mse: 150738092032.0000 - val_l

231/231 [==============================] - 0s 1ms/step - loss: 133984354304.0000 - mse: 133984354304.0000 - val_loss: 72730877952.0000 - val_mse: 72730877952.0000
Epoch 58/70
231/231 [==============================] - 0s 1ms/step - loss: 140922339328.0000 - mse: 140922339328.0000 - val_loss: 69760016384.0000 - val_mse: 69760016384.0000
Epoch 59/70
231/231 [==============================] - 0s 1ms/step - loss: 143368437760.0000 - mse: 143368437760.0000 - val_loss: 79892799488.0000 - val_mse: 79892799488.0000
Epoch 60/70
231/231 [==============================] - 0s 1ms/step - loss: 140091080704.0000 - mse: 140091080704.0000 - val_loss: 64653320192.0000 - val_mse: 64653320192.0000
Epoch 61/70
231/231 [==============================] - 0s 1ms/step - loss: 147365085184.0000 - mse: 147365085184.0000 - val_loss: 78726750208.0000 - val_mse: 78726750208.0000
Epoch 62/70
231/231 [==============================] - 0s 1ms/step - loss: 144651485184.0000 - mse: 144651485184.0000 - val_loss: 8146322

...

Epoch 107/110
231/231 [==============================] - 0s 2ms/step - loss: 89137750016.0000 - mse: 89137750016.0000 - val_loss: 52032180224.0000 - val_mse: 52032180224.0000
Epoch 108/110
231/231 [==============================] - 0s 2ms/step - loss: 91077976064.0000 - mse: 91077976064.0000 - val_loss: 37871783936.0000 - val_mse: 37871783936.0000
Epoch 109/110
231/231 [==============================] - 0s 2ms/step - loss: 90619068416.0000 - mse: 90619068416.0000 - val_loss: 36163194880.0000 - val_mse: 36163194880.0000
Epoch 110/110
116/116 [==============================] - 0s 642us/step - loss: 90333224960.0000 - mse: 90333224960.0000
Epoch 1/110


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 12.4min finished


346/346 [==============================] - 0s 1ms/step - loss: 114424733696.0000 - mse: 114424733696.0000 - val_loss: 66982805504.0000 - val_mse: 66982805504.0000
Epoch 2/110
346/346 [==============================] - 0s 863us/step - loss: 80023306240.0000 - mse: 80023306240.0000 - val_loss: 66118844416.0000 - val_mse: 66118844416.0000
Epoch 3/110
346/346 [==============================] - 0s 849us/step - loss: 78973943808.0000 - mse: 78973943808.0000 - val_loss: 65565220864.0000 - val_mse: 65565220864.0000
Epoch 4/110
346/346 [==============================] - 0s 833us/step - loss: 78269775872.0000 - mse: 78269775872.0000 - val_loss: 65087795200.0000 - val_mse: 65087795200.0000
Epoch 5/110
346/346 [==============================] - 0s 829us/step - loss: 77750755328.0000 - mse: 77750755328.0000 - val_loss: 66304503808.0000 - val_mse: 66304503808.0000
Epoch 6/110
346/346 [==============================] - 0s 833us/step - loss: 78047174656.0000 - mse: 78047174656.0000 - val_loss: 6439663

Epoch 48/110
346/346 [==============================] - 0s 788us/step - loss: 58105618432.0000 - mse: 58105618432.0000 - val_loss: 45919432704.0000 - val_mse: 45919432704.0000
Epoch 49/110
346/346 [==============================] - 0s 788us/step - loss: 58063941632.0000 - mse: 58063941632.0000 - val_loss: 48326770688.0000 - val_mse: 48326770688.0000
Epoch 50/110
346/346 [==============================] - 0s 792us/step - loss: 55297327104.0000 - mse: 55297327104.0000 - val_loss: 50182574080.0000 - val_mse: 50182574080.0000
Epoch 51/110
346/346 [==============================] - 0s 826us/step - loss: 54829555712.0000 - mse: 54829555712.0000 - val_loss: 54487801856.0000 - val_mse: 54487801856.0000
Epoch 52/110
346/346 [==============================] - 0s 842us/step - loss: 56245817344.0000 - mse: 56245817344.0000 - val_loss: 41801560064.0000 - val_mse: 41801560064.0000
Epoch 53/110
346/346 [==============================] - 0s 791us/step - loss: 56406102016.0000 - mse: 56406102016.0000 -

346/346 [==============================] - 0s 816us/step - loss: 48183582720.0000 - mse: 48183582720.0000 - val_loss: 29013788672.0000 - val_mse: 29013788672.0000
Epoch 95/110
346/346 [==============================] - 0s 791us/step - loss: 48497958912.0000 - mse: 48497958912.0000 - val_loss: 38235439104.0000 - val_mse: 38235439104.0000
Epoch 96/110
346/346 [==============================] - 0s 822us/step - loss: 44962721792.0000 - mse: 44962721792.0000 - val_loss: 29728004096.0000 - val_mse: 29728004096.0000
Epoch 97/110
346/346 [==============================] - 0s 821us/step - loss: 43983679488.0000 - mse: 43983679488.0000 - val_loss: 47665889280.0000 - val_mse: 47665889280.0000
Epoch 98/110
346/346 [==============================] - 0s 812us/step - loss: 45428666368.0000 - mse: 45428666368.0000 - val_loss: 35028848640.0000 - val_mse: 35028848640.0000
Epoch 99/110
346/346 [==============================] - 0s 819us/step - loss: 46432960512.0000 - mse: 46432960512.0000 - val_loss: 32

In [18]:
#getting the best parameters
history_3.best_params_

{'optimizer': 'RMSprop',
 'num_layers': 6,
 'neurons': 56,
 'learning_rate': 0.03,
 'epochs': 110,
 'drop_out': 0.2}

In [19]:
#checking the RMSE
np.sqrt(abs(history_3.best_score_))

194010.88475306396

In [20]:
model_3 = create_model(optimizer = 'RMSprop', num_layers = 6, neurons = 56, drop_out = .2, learning_rate= .03)

In [21]:
history_3 = model_3.fit(X_train, y_train, epochs = 110, validation_data=(X_valid, y_valid),
                       callbacks = [tf.keras.callbacks.EarlyStopping(patience = 20)])

Epoch 1/110
346/346 [==============================] - 0s 1ms/step - loss: 113054523392.0000 - mse: 113054523392.0000 - val_loss: 66642526208.0000 - val_mse: 66642526208.0000
Epoch 2/110
346/346 [==============================] - 0s 845us/step - loss: 79559548928.0000 - mse: 79559548928.0000 - val_loss: 67380391936.0000 - val_mse: 67380391936.0000
Epoch 3/110
346/346 [==============================] - 0s 814us/step - loss: 78252040192.0000 - mse: 78252040192.0000 - val_loss: 65369600000.0000 - val_mse: 65369600000.0000
Epoch 4/110
346/346 [==============================] - 0s 832us/step - loss: 76729057280.0000 - mse: 76729057280.0000 - val_loss: 64838258688.0000 - val_mse: 64838258688.0000
Epoch 5/110
346/346 [==============================] - 0s 860us/step - loss: 76393537536.0000 - mse: 76393537536.0000 - val_loss: 65767911424.0000 - val_mse: 65767911424.0000
Epoch 6/110
346/346 [==============================] - 0s 823us/step - loss: 76855869440.0000 - mse: 76855869440.0000 - val_l

346/346 [==============================] - 0s 822us/step - loss: 55074717696.0000 - mse: 55074717696.0000 - val_loss: 44395634688.0000 - val_mse: 44395634688.0000
Epoch 48/110
346/346 [==============================] - 0s 814us/step - loss: 55551504384.0000 - mse: 55551504384.0000 - val_loss: 59245588480.0000 - val_mse: 59245588480.0000
Epoch 49/110
346/346 [==============================] - 0s 806us/step - loss: 55644102656.0000 - mse: 55644102656.0000 - val_loss: 41486159872.0000 - val_mse: 41486159872.0000
Epoch 50/110
346/346 [==============================] - 0s 829us/step - loss: 54050287616.0000 - mse: 54050287616.0000 - val_loss: 48110563328.0000 - val_mse: 48110563328.0000
Epoch 51/110
346/346 [==============================] - 0s 828us/step - loss: 52984041472.0000 - mse: 52984041472.0000 - val_loss: 68873748480.0000 - val_mse: 68873748480.0000
Epoch 52/110
346/346 [==============================] - 0s 838us/step - loss: 54052548608.0000 - mse: 54052548608.0000 - val_loss: 38

346/346 [==============================] - 0s 840us/step - loss: 44017958912.0000 - mse: 44017958912.0000 - val_loss: 42058498048.0000 - val_mse: 42058498048.0000
Epoch 94/110
346/346 [==============================] - 0s 841us/step - loss: 45931429888.0000 - mse: 45931429888.0000 - val_loss: 30391320576.0000 - val_mse: 30391320576.0000
Epoch 95/110
346/346 [==============================] - 0s 829us/step - loss: 46681296896.0000 - mse: 46681296896.0000 - val_loss: 35411120128.0000 - val_mse: 35411120128.0000
Epoch 96/110
346/346 [==============================] - 0s 841us/step - loss: 45357572096.0000 - mse: 45357572096.0000 - val_loss: 27834361856.0000 - val_mse: 27834361856.0000
Epoch 97/110
346/346 [==============================] - 0s 838us/step - loss: 42773737472.0000 - mse: 42773737472.0000 - val_loss: 48449175552.0000 - val_mse: 48449175552.0000
Epoch 98/110
346/346 [==============================] - 0s 842us/step - loss: 44358819840.0000 - mse: 44358819840.0000 - val_loss: 34

In [22]:
print('Evaluation at Epoch 110:\n')
for key in history_3.history:
    print(key + ':' + str(history_3.history[key][-1]) + '\n')
    
    
print('RMSE :' + str(np.sqrt(history_3.history['val_mse'][-1])))

Evaluation at Epoch 110:

loss:43033812992.0

mse:43033812992.0

val_loss:28424933376.0

val_mse:28424933376.0

RMSE :168596.95541735028


This here gives us the lowest rmse in terms of validation scores. Compared to the other models, they all seem to perform better in terms of minimum error achieved. I probably have to do further tinkering of specific parameters to lower the error to compete with the other models.  